Loading dataset

In [18]:
import pandas as pd

In [25]:
headlines = pd.read_json(r'C:\Users\jakub.kalka\Downloads\Graduate - HEADLINES dataset (2019-06) (1).json', lines=True)

In [26]:
headlines.head()

,headline,is_sarcastic
0,former versace store clerk sues over secret 'b...,0
1,the 'roseanne' revival catches up to our thorn...,0
2,mom starting to fear son's web series closest ...,1
3,"boehner just wants wife to listen, not come up...",1
4,j.k. rowling wishes snape happy birthday in th...,0


In [34]:
headlines.dtypes

headline        object
is_sarcastic     int64
dtype: object

For NLP process important thing is clearing text, and coding to usable form. One of this is Stemming. 
Stemming is the process of reducing a word to its word stem that affixes to suffixes and prefixes 
or the roots of words known as a lemma.
Next I use TD_IDF (Term Frequency-Inverse Document). It's counting frequency of words in whole dataset.

We also can use StopWords(Remove words like and, to, that etc), but in this case I don't use StopWords.

cleaning data

In [36]:
import re 
headlines['headline'] = headlines['headline'].apply(lambda s : re.sub('[^a-zA-Z]', ' ', s))

In [52]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
stemm_headlines = headlines['headline'].apply(lambda x: x.split())
stemm_headlines = headlines['headline'].apply(lambda x : ''.join([ps.stem(word) for word in x]))


In [57]:
from sklearn.feature_extraction.text import TfidfVectorizer
tv = TfidfVectorizer(max_features = 1000)
stemm_headlines = list(stemm_headlines)
stemm_headlines = tv.fit_transform(stemm_headlines).toarray()

In [64]:
from sklearn.model_selection import train_test_split
headline_train, headline_test, labels_train, labels_test = train_test_split(stemm_headlines, headlines['is_sarcastic'], test_size = 0.2, random_state = 0)

In [70]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
lr = LogisticRegression()
lr.fit(headline_train, labels_train)
lr_accuracy = lr.score(headline_test, labels_test)

c:\python\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [72]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators = 10, random_state = 0)
rf.fit(headline_train, labels_train)
rf_accuracy = rf.score(headline_test, labels_test)


In [73]:
from sklearn.svm import LinearSVC
svc = LinearSVC()
svc.fit(headline_train, labels_train)
svc_accuracy = svc.score(headline_test, labels_test)

In [74]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(headline_train, labels_train) 
gnb_accuracy = gnb.score(headline_test, labels_test)

In [75]:
compare_models = pd.DataFrame({
    'GNB': [gnb_accuracy],
    'SVC': [svc_accuracy],
    'Logistic Regression': [lr_accuracy],
    'Random Forest': [rf_accuracy]    
})
compare_models

,GNB,SVC,Logistic Regression,Random Forest
0,0.726694,0.795582,0.802508,0.777611


Model with the best accuracy is Logistic Regresion. This model should be use to get new predictons. For a better model may have been used other classifaction model. Can focus more attention to pre processing text, for example use stop words.